In [1]:
from sae import TopKSAE, TopKSAEConfig
from generate_residuals import EmbeddingGeneratorConfig


import torch
import os
from tqdm import tqdm
from matplotlib import pyplot as plt

device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"
torch.autograd.set_grad_enabled(False)


In [33]:
# s = torch.load('./saes/sae_no_latent_bias.pt')
s = torch.load('./saes/sae_small.pt')
sae = TopKSAE(s['config'])
sae.load_state_dict(s['model'])

<All keys matched successfully>

In [34]:
sae

TopKSAE(
  (encode): Linear(in_features=384, out_features=32768, bias=True)
  (decode): Linear(in_features=32768, out_features=384, bias=False)
)

In [35]:
embdconfig = torch.load("./small_residuals/0.pt")['config']

In [36]:
embdconfig

EmbeddingGeneratorConfig(batch_size=512, block_size=512, n_embd=384, ratio_tokens_saved=0.07, residual_layer=8, mb_per_save=2000, save_dir='./small_residuals')

In [37]:
residuals_dir = embdconfig.save_dir

residuals_files = [os.path.join(residuals_dir, f) for f in os.listdir(residuals_dir)]

In [38]:
residuals_files[0]

'./small_residuals/12.pt'

In [39]:
residual = torch.load(residuals_files[0])

In [40]:
residual.keys() # dict_keys(['residuals', 'token_idxs', 'context_window_starts', 'config'])


dict_keys(['residuals', 'token_idxs', 'token_values', 'context_window_starts', 'config'])

In [41]:
sae.config

TopKSAEConfig(embedding_size=384, n_features=32768, topk=24, lr=0.001, batch_size=4096, latent_bias=True)

In [42]:
def crop_fit_batch(tensor, batch_size):
    n = tensor.size(0)
    n_batches = n // batch_size
    n = n_batches * batch_size
    return tensor[:n]

In [43]:


def sae_process_residuals(residuals_path, sae):
    topk_idxs_per_token = []
    topk_strengths_per_token = []

    residual = torch.load(residuals_path)
    residuals = residual['residuals'].to(torch.float32)
    token_idxs = residual['token_idxs']
    context_window_starts = residual['context_window_starts']
    config = residual['config']
    sae.eval()
    cropped = crop_fit_batch(residuals, sae.config.batch_size)
    batches = cropped.view(-1, sae.config.batch_size, sae.config.embedding_size)
    with torch.no_grad():
        for batch in tqdm(batches):
            sae_out = sae(batch)
            topk_idxs = sae_out['topk_idxs']
            topk_values = sae_out['topk_values']
            topk_idxs_per_token.append(topk_idxs)
            topk_strengths_per_token.append(topk_values)
        topk_idxs = torch.cat(topk_idxs_per_token, dim=0)
        topk_strengths = torch.cat(topk_strengths_per_token, dim=0)
    return topk_idxs, topk_strengths, token_idxs, context_window_starts


topk_idxs_per_token, topk_strengths_per_token, dataset_token_location_idxs, dataset_context_window_starts = sae_process_residuals(residuals_files[-1], sae)

100%|██████████| 636/636 [00:04<00:00, 153.40it/s]


In [46]:
topk_idxs_per_token[7]

tensor([28725, 17068,  6470,  8775,    12, 28371, 18882, 20139, 25526, 23063,
         6672, 10003,  3065, 20227,  2933, 25137, 10464, 22462, 18098, 21377,
         1265,  7810, 19975, 30799], device='cuda:0')

In [23]:
top_features = topk_idxs_per_token[:, 0].view(-1)
unique, counts = torch.unique(top_features, return_counts=True)

In [24]:
torch.numel(top_features)

2605056

In [50]:
sae

TopKSAE(
  (encode): Linear(in_features=512, out_features=32768, bias=False)
  (decode): Linear(in_features=32768, out_features=512, bias=False)
)

In [25]:
torch.sort(counts, descending=True)[0][0:10]/torch.numel(top_features)

tensor([8.2767e-01, 1.7233e-01, 3.8387e-07, 3.8387e-07], device='cuda:0')

In [ ]:
dataset_context_window_starts = torch.tensor(dataset_context_window_starts)
dataset_token_location_idxs = torch.tensor(dataset_token_location_idxs)

sort_idxs = torch.argsort(dataset_token_location_idxs)
sorted_context_window_starts = dataset_context_window_starts[sort_idxs]
sorted_token_idxs = dataset_token_location_idxs[sort_idxs]

In [ ]:
idx = 55
sorted_context_window_starts[idx], sorted_token_idxs[idx]

(tensor(512, device='cuda:0'), tensor(795, device='cuda:0'))

In [ ]:
random_feature = topk_idxs_per_token.view(-1)[6009]

flat_topk_idxs = topk_idxs_per_token.view(-1)

random_feature_indexes = torch.where(flat_topk_idxs == random_feature)[0]
random_feature_strengths = topk_strengths_per_token.view(-1)[random_feature_indexes]

random_feature_subset_strengths, random_feature_subset_flat_feature_idxs = random_feature_strengths.topk(10)
# the indexes are relative to the random_feature_indexes(subset of data), we need to map back to the global indexes
random_feature_subset_flat_feature_idxs = random_feature_indexes[random_feature_subset_flat_feature_idxs]

# random_feature_subset_location_idx
random_feature_subset_token_idxs = random_feature_subset_flat_feature_idxs // sae.config.topk
random_feature_subset_topk_idxs = random_feature_subset_flat_feature_idxs % sae.config.topk

print("selected feature:", random_feature)

token_idx = random_feature_subset_token_idxs[1]
topk_idx = random_feature_subset_topk_idxs[1]

topk_idxs_per_token[token_idx, topk_idx] # as expected, the selected feature via lookup is the same as the random feature

selected feature: tensor(20184, device='cuda:0')


tensor(20184, device='cuda:0')

In [ ]:
from gpt import preprocess_tokens_from_huggingface
import transformers


enc = transformers.AutoTokenizer.from_pretrained('activated-ai/tiny-stories-8k-tokenizer')
preprocess_tokens_from_huggingface("./datasets")


train =  torch.load("datasets/train.pt", map_location=device).to(torch.int64)


skipping token preprocessing for validation : using cache ./datasets/validation.pt
skipping token preprocessing for train : using cache ./datasets/train.pt


In [ ]:
def get_context(token_idx):
    start = token_idx - 10
    end = token_idx + 3
    return enc.decode(train[start:token_idx])+"<"+enc.decode(train[token_idx])+">"+enc.decode(train[token_idx+1:end])



for token_idx in random_feature_subset_token_idxs:
    token_idx = token_idx.item()
    dataset_token_idx = dataset_token_location_idxs[token_idx]
    print(get_context(dataset_token_idx))
    print("-"*10)


 each other but with my pistol!" The other friend< agreed>. They
----------
. "It will be fun!"

Ben< agreed>. They
----------
 that they should go on an adventure. The squirrel< agreed>, so
----------
 fight! It will be so much fun!" Jo< agreed>.

----------
 go on an adventure!â€ Celery nodded< in> agreement and
----------
 see a movie," she said. Ben nodded his< head> and smiled
----------
 some more vegetables! They'll be cheap!" Peter< agreed>, so
----------
ggy, let us start!"
Iggy barked< in> excitement and
----------
s go explore the wild!â€ Jim agreed<,> and so
----------
 them to different places." Ben says, "OK<,> that sounds
----------
